In [1]:
'''
Title: Classifying Congressional Tweets Using Various Machine Learning Algorithms

Description:

I am trying to read tweets written by Congresspeople (Senate and House) and use machine learning to 
predict what party they're from. The tweets are collected using Twitter's API and the python library
Tweepy. I'm converting the text of the words to a bag of words, filtering out punctuation and stemming 
as well. I'm leaving in hashtags and @s. 

In terms of specific algorithms, I expect to use multiple different classifiers. I also will use a neural net,
though I'm not sure the architecture I'll aim for. I still need to do more research on what's commonly
used for NLP. It will likely incorporate existing sentiment analysis models to improve accuracy. 
In addition to that, I'll try a number of different classifiers, using grid search to optimize them, 
and test them with both test set accuracy and k-fold cross validation.

I plan to present the best models based on multiple metrics, including accuracy, time to learn, and time to
classify. I'll include the optimized parameters as well, and the optimal preparatory pipelines.

I've done most of the data preprocessing and collected a decent-sized dataset. I've run a logistic
regression classifier with default hyperparameters and a tfidf vectorizer. Stemming and filtering functions
are completed. You can see the accuracy of this basic model below; with 75% accuracy on the test set,
I'd say that's a good proof of concept.

I still need to complete my other models, and I also would like to get a larger dataset. I need to see how to
get more tweets per person using tweepy. I'll also look into other tools for preprocessing. And I'll
look into other dictionaries that could be helpful.

'''

"\nTitle: Classifying Congressional Tweets Using Various Machine Learning Algorithms\n\nDescription:\n\nI am trying to read tweets written by Congresspeople (Senate and House) and use machine learning to \npredict what party they're from. The tweets are collected using Twitter's API and the python library\nTweepy. I'm converting the text of the words to a bag of words, filtering out punctuation and stemming \nas well. I'm leaving in hashtags and @s. \n\nIn terms of specific algorithms, I expect to use multiple different classifiers. I also will use a neural net,\nthough I'm not sure the architecture I'll aim for. I still need to do more research on what's commonly\nused for NLP. It will likely incorporate existing sentiment analysis models to improve accuracy. \nIn addition to that, I'll try a number of different classifiers, using grid search to optimize them, \nand test them with both test set accuracy and k-fold cross validation.\n\nI plan to present the best models based on multipl

In [2]:
#test some stuff

import tweepy

auth = tweepy.AppAuthHandler("Y49bfRjEyvHDa0vUBRFC4oQJy", "CCrNSKGS1zVvYBCGJS1bMFji2qkuoTujGUqZZUbWV2QEl1QtCE")
api = tweepy.API(auth)
print(api.get_status(1324381354453250049, tweet_mode = "extended").full_text[0:-24])
print(api.get_status(1324381354453250049, tweet_mode = "extended").retweeted)


I’m holding a press conference with Dr. Scott Harris today to provide an update on our state’s #COVID19 efforts. You can watch it live at 11 a.m. via https://t.co/Y9juKjW5io or your local news stations. #alpolitics #TogetherAL @ALPublicHealth
False


In [3]:
# get names, parties, clean @s

import pandas as pd 
import numpy as np
dataset = pd.read_csv('handledatasetwithparty.csv', header=None)
dataset[3].unique()

array = dataset.to_numpy()
independents = array[array[:,3]=='I']
democrats = array[array[:,3]=='D']
republicans = array[array[:,3]=='R']
libertarians = array[array[:,3]=='L']
# other = array[array[:,3] not in ['I','D','R','L']] # doesn't work
other = array[(array[:,3]!='I') & (array[:,3]!='D') & (array[:,3]!='R') & (array[:,3]!='L')]

a = np.concatenate((republicans, democrats))
a = a[:, [3,4]]
# print np.where(array1==0, 1, array1) 
print(a[1][1])
print(np.any(a=='@RepFredKelle'))
print(np.any(a=='\xa0@SenatorLeahy'))
print(np.any(a=='@RepLloyd'))
a = np.where(a=='@RepLloyd', '@RepLloydDoggett', a)
a = np.where(a=='@RepFredKelle', '@RepFredKeller', a)
a = np.where(a=='\xa0@SenatorLeahy', '@SenatorLeahy', a)
print(np.any(a=='@RepLloyd'))
print(np.any(a=='xa0@SenatorLeahy'))
print(np.any(a=='@RepFredKelle'))

a
index = np.where(a=="@RepRickCrawford")
index
a[18]

@RepByrne
True
True
True
False
False
False


array(['R', '@RepRickCrawford'], dtype=object)

In [9]:
#get tweets. Instead of running this, can run cell 2 below instead

import tweepy

auth = tweepy.AppAuthHandler("Y49bfRjEyvHDa0vUBRFC4oQJy", "CCrNSKGS1zVvYBCGJS1bMFji2qkuoTujGUqZZUbWV2QEl1QtCE")
api = tweepy.API(auth)
failures = []
tmp=[]  
tmp2 = np.zeros(len(a))


def get_tweets(party, username): 
    tweepy.Cursor(api.user_timeline, id="twitter")


  
    # 200 tweets to be extracted 
#     number_of_tweets=200
#     try:
#     tweets = api.user_timeline(screen_name=username, tweet_mode="extended") 
    for j in tweepy.Cursor(api.user_timeline, screen_name=username, tweet_mode="extended", wait_on_rate_limit= True,
                           retry_count=10, retry_delay=5, retry_errors=set([503])

).items():
#         pass
#     tweets = [tweet for tweet in tweets] # CSV file created  
#     for j in tweets: 

        # Appending tweets to the empty array tmp 
        text = j.full_text[0:-24]
        if(j.created_at.year != 2020):
            break
        if text[0:4] != 'RT @':
            tmp.append([j.user._json['screen_name'],text,party])  
            print(j.user._json['screen_name'])
            print(j.full_text[0:-24])
            print(j.created_at.year)
#     except:
#         print('not authorized: ' + username)
#         failures.append(username)

for user_id in a:
# user_id = ['R', "@RepRickCrawford"]
    get_tweets(user_id[0], user_id[1])
# tmp

# TODO: REMOVE BELOW
fulldf = pd.DataFrame(tmp, columns=["account","text","party"])
fulldf.to_csv("fulldataset.csv")

SenShelby
Today is #SmallBusinessSaturday! I encourage all Alabamians to #ShopSmall &amp; support the local businesses that are the economic engine of our state. Many businesses have options in place for safe shopping. Check with your local stores to see what responsible services they offer.
2020
SenShelby
Wishing everyone a very happy #Thanksgiving. Today we remember what we're grateful for, such as family, friends, and freedom. I am also thankful for our first responders and military personnel who protect our liberties and freedoms today and every day.
2020
SenShelby
The @USAirForce has selected Maxwell Air Force Base in #Montgomery to host the new MH-139 Grey Wolf helicopter. The decision highlights and reinforces our state's success in supporting a robust national defense. This is excellent news for #Alabama.
2020
SenShelby
Great news! @NSF recently awarded @SamfordU $1 million in grant funding aimed to boost #STEM opportunities for community college transfers. This award will esta

SenShelby
.@fema recently approved AL’s disaster declaration for Hurricane #Sally. Federal funding to aid in the recovery efforts is now available to affected individuals in Baldwin, Escambia, and Mobile Counties. Assistance can be applied for online:
2020
SenShelby
As flooding persists throughout Alabama, please continue to listen to local officials and take the proper precautionary measures in the coming days. Additionally, the following link includes information from @fema related to flood damaged property:
2020
SenShelby
Important announcement for Alabama’s agriculture industry. Thank you, @USDA!
2020
SenShelby
As Hurricane #Sally makes its way inland, please continue to listen to local officials, monitor the weather for your area closely, and stay off the roadways if at all possible to allow first responders to reach those in need.
2020
SenShelby
Please visit my website for additional links to resources related to Hurricane #Sally: https://t.co/BzNm072mwG
2020
SenShelby
As Hurrica

SenShelby
SiO2 Materials Science in #Auburn is receiving $143M to accelerate production of their packaging platform for storing #coronavirus vaccines &amp; therapeutics - adding 200 skilled manufacturing jobs. Proud this #Alabama company is working to combat #COVID19.
2020
SenShelby
Today marks 76 years since American troops stormed the beaches of Normandy. We owe our utmost gratitude to the Greatest Generation as we remember and honor those who fought for our freedom. #DDay76
2020
SenShelby
According to @USTreasury, 60,909 in #Alabama have received #PPP loans totaling $6,112,333,061 through May 30th. The Paycheck Protection Program is clearly working in our state &amp; throughout the nation. Proud more relief i
2020
SenShelby
Today @POTUS signed the #PaycheckProtectionProgram Flexibility Act – providing relief for #smallbiz. The bill allows businesses granted loan forgiveness to receive payroll tax deferment, extends deadline for #smallbiz to rehire workers receiving unemployme
2020
S

SenShelby
Today @NOAA announced their plan to provide $300 million to assist the U.S. fishing industry in mitigating impacts caused by #COVID19. This includes $3.3M for #Alabama! I am proud to have fought to include this f
2020
SenShelby
While it’s not a normal day in the Senate, I am still hosting my Republican colleagues for a social distancing lunch. My gift to them includes @JimnNicksBBQ cheese biscuit mix and some #Alabama pottery from @AlabamaGoods. #MadeinAlabama
2020
SenShelby
Today the Senate @BankingGOP Committee held a hearing to consider the nomination of Dana Wade to be @FHAgov commissioner. Her prior public service &amp; extensive knowledge of housing issues make her well-s
2020
SenShelby
Today the @AlabamaNG will conduct a flyover to honor healthcare workers, first responders, and other essential employees for their efforts during the #COVID19 crisis. Proud these hardworking Alabamians are being recognized and commended!
2020
SenShelby
Happy #TeacherAppreciationWeek to a

KeyboardInterrupt: 

In [6]:
len(tmp)

15312

In [5]:
#convert to df

origdf = pd.DataFrame(tmp, columns=["account","text","party"])
origdf.to_csv("fulldataset.csv")
origdf

,account,text,party
0,SenShelby,Today is #SmallBusinessSaturday! I encourage a...,R
1,SenShelby,Wishing everyone a very happy #Thanksgiving. T...,R
2,SenShelby,The @USAirForce has selected Maxwell Air Force...,R
3,SenShelby,Great news! @NSF recently awarded @SamfordU $1...,R
4,SenShelby,Thank you to each and every one of the brave m...,R
...,...,...,...
8893,RepGwenMoore,It hurt my heart to learn of the shooting at M...,D
8894,RepGwenMoore,"In a year of records, 2020 has unfortunately s...",D
8895,RepGwenMoore,These aid programs assist businesses and local...,D
8896,RepGwenMoore,Essential workers are not expendable. And corp...,D


In [20]:
origdf = pd.read_csv('origdataset.csv')
origdf

,Unnamed: 0,account,text,party
0,0,SenShelby,Today is #SmallBusinessSaturday! I encourage a...,R
1,1,SenShelby,Wishing everyone a very happy #Thanksgiving. T...,R
2,2,SenShelby,The @USAirForce has selected Maxwell Air Force...,R
3,3,SenShelby,Great news! @NSF recently awarded @SamfordU $1...,R
4,4,SenShelby,Thank you to each and every one of the brave m...,R
...,...,...,...,...
8893,8893,RepGwenMoore,It hurt my heart to learn of the shooting at M...,D
8894,8894,RepGwenMoore,"In a year of records, 2020 has unfortunately s...",D
8895,8895,RepGwenMoore,These aid programs assist businesses and local...,D
8896,8896,RepGwenMoore,Essential workers are not expendable. And corp...,D


In [7]:
import nltk
nltk.download('stopwords')

len(tmp)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\quincy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


8898

In [10]:
# process text
import string

removable_punctuation = "!\"$%&'()*+,-./:;<=>?[\]^_`{|}~"
translator = str.maketrans('', '', removable_punctuation)

def remove_links(text):
    index = text.find('https://t.co/')
    if index is -1:
        return text
    else:
        text = text[0:index] + text[index+23:]
        return remove_links(text)
 

def preprocessor(text):
    try:
        text = remove_links(text)
#         print(text[index+23:])
    except:
        print(text)
    text = text.replace("&amp;", '').lower()
    return text.translate(translator)
    
# preprocessor(df.iloc[0]['text'])
df = origdf.copy()
df['text'] = df['text'].apply(preprocessor)

df

nan


AttributeError: 'float' object has no attribute 'replace'

In [18]:
df[df['text'].isnull()]

,Unnamed: 0,account,text,party
142,142,lisamurkowski,NaN,R
168,168,RepAmata,NaN,R
170,170,RepAmata,NaN,R
173,173,RepAmata,NaN,R
177,177,RepAmata,NaN,R
...,...,...,...,...
8353,8353,RepEBJ,NaN,D
8355,8355,RepEBJ,NaN,D
8631,8631,StaceyPlaskett,NaN,D
8634,8634,PattyMurray,NaN,D


In [ ]:
#tokenizer

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()

def tokenizer(text):
    return [porter.stem(word) for word in text.split() if word not in stopwords.words('english')]



In [ ]:
#simple logistic regression

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

X = list(df['text'])
y = list(df['party'])
X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size = 0.3, random_state = 0)

lr = Pipeline([('vect', tfidf),('clf', LogisticRegression(random_state=0, solver='liblinear'))])

lr.fit(X_train,y_train)
results_train = lr.predict(X_train) == y_train
print("Accuracy on training set:")
print(np.count_nonzero(results_train)/len(results_train))
results_test = lr.predict(X_test) == y_test
print("Accuracy on test set:")
print(np.count_nonzero(results_test)/len(results_test))